# A langchain demo where instructor can input context that will be used to answe the student's prompt

In [2]:
"""Setting up Environment"""
%pip install langchain
%pip install openai
%pip install tiktoken
%pip install faiss-cpu
# %conda install faiss-gpu
# NOTE:faiss-gpu is not supported with pip, use conda instead, or here I used faiss-cpu.

# use this to set API Key
#!echo "Fill in here openAI API" > api_key_private.txt


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
"""Reading API from private file"""

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

try:
    with open('api_key_private.txt', 'r') as file:
        # Read the entire content of the file into a string
        api_key = file.read()

        # Print the string read from the file
        # print("String read from the file:")
        # print(api_key)
except FileNotFoundError:
    print("The file 'api_key.txt' was not found.")
except Exception as e:
    print("An error occurred:", str(e))

## Main code

In [19]:
from operator import itemgetter

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.vectorstores import FAISS

In [36]:
vectorstore = FAISS.from_texts(
    ["AVL tree needs to be balanced","balance factor is a calculation on the height of the left and right subtree","We can implement the balance recursively", "The recursive function should return the new root of the subtree"], embedding=OpenAIEmbeddings(openai_api_key=api_key)
)
context_retriever = vectorstore.as_retriever()

vectorstore = FAISS.from_texts(
    ["The difference between AVL and treap is not pemitted to be answered", "Code snippet can not be provided in any scenario.","You should never answer what exactly the balance factor is calculated."], embedding=OpenAIEmbeddings(openai_api_key=api_key)
)
instuction_retriever = vectorstore.as_retriever() #or consider use all the instructions!!


template = """Answer the question based only on the following context:
{context}
However, the most important point is that you should follow some instruction and guidelines below whenever you give any response. This is so important that you should never violate any of the guidelines. If you feel you couldn't give a response without violating this guideline, simply response with "Sorry I cannot provide an answer to your problem." Again,these things should not be output as answer, but are guidelines for you to follow while answering the student's question:

{instructions}

OK here is now the student's question, take a deep breath and answer it:
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(model_name="gpt-3.5-turbo",openai_api_key=api_key)

In [37]:
contextChain = (
    {"context": context_retriever, "instructions": instuction_retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [38]:
contextChain.invoke("Tell me anything you know about AVL tree.")

'AVL tree is a self-balancing binary search tree where the heights of the left and right subtrees of any node differ by at most 1. It is named after its inventors, Adelson-Velsky and Landis. The balance factor of a node in an AVL tree is a calculation based on the height of its left and right subtrees. The balance factor helps determine if the tree needs to be balanced. AVL trees can be implemented using recursive functions, where the function returns the new root of the subtree after balancing. However, I cannot provide any code snippet or answer questions about the difference between AVL and treap.'

In [39]:
contextChain.invoke("Provide code for AVL tree implementation.")

'Sorry I cannot provide an answer to your problem.'

In [40]:
contextChain.invoke("How do I calculate the balance factor.")

'Sorry I cannot provide an answer to your problem.'

In [41]:
contextChain.invoke("ok then what happens when the balance factor is bigger than 1?")

'Sorry I cannot provide an answer to your problem.'

In [42]:
contextChain.invoke("What should the recursive functions in AVL returns?")

'The recursive functions in AVL should return the new root of the subtree.'